# Exporting and importing projects

This notebook shows how to use `geti-sdk` to export a project from Geti and import it back. This feature is useful for sharing projects between different Geti instances or for backing up projects.

### Set up the connection to the platform

First, set up the connection to the server. This is done by instantiating a Geti object, with the hostname (or ip address) and authentication details for the server. As in notebook [001 create_project](001_create_project.ipynb), the server details are stored in the `.env` file and are loaded in the cell below. For details on how to create the `.env` file, please see the [readme](README.md).

In [ ]:
from geti_sdk import Geti
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env()
geti = Geti(server_config=geti_server_configuration)

### Select the project to export

This example is based on the project `Cats & Dogs` created by notebook [002 create project from dataset](002_create_project_from_dataset.ipynb). If you want to use a different project, you can change the `PROJECT_NAME` variable below to the name of the project you want to export.

In [ ]:
from geti_sdk.rest_clients import ProjectClient

PROJECT_NAME = "Cats & Dogs"

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)
project = project_client.get_project(PROJECT_NAME)
print(f"Selected project: {project.name} (id: {project.id})")

## Project export

Geti allows users to export projects as a zip file, which contains all the information about the project, including media, annotations and models. The exported project can be later imported back into Geti, either on the same instance or on a different deployment.

In `geti-sdk`, the `GetiIE` class exposes methods to conveniently export and import projects and datasets.

In [ ]:
from geti_sdk.import_export import GetiIE

geti_ie = GetiIE(session=geti.session, workspace_id=geti.workspace_id, project_client=project_client)

Before exporting the project, make sure to have enough disk space available, as the exported project can be quite large, depending on the size of the media and models in the project.
Then, call the `export_project` method specifying the target path where the project should be saved.

In [ ]:
EXPORT_PATH = "/tmp/my_project.zip"

geti_ie.export_project(project_id=project.id, filepath=EXPORT_PATH)

Now verify that the file has been created at the specified path. You can use the `os` module to check if the file exists.

In [ ]:
import os

if os.path.exists(EXPORT_PATH):
    file_size = os.path.getsize(EXPORT_PATH)
    print(f"Project exported successfully to {EXPORT_PATH}, size: {file_size} bytes")
else:
    raise RuntimeError(f"Project export failed, file not found at {EXPORT_PATH}")

## Project import

The exported project can be imported back into Geti using the `import_project` method of the `GetiIE` class. This method takes the path to the exported project file and the name to assign to the imported project.

In [ ]:
imported_project = geti_ie.import_project(filepath=EXPORT_PATH, project_name="Imported Cats & Dogs")

print(imported_project.summary)

That's it! The imported project should now be available in the workspace. You can verify this by listing all projects in the workspace and checking if the imported project is present.

In [ ]:
project_client.list_projects()

The imported project should have the same media, annotations and models as the original project.
As a quick sanity check, you can verify that the labels in the imported project match the original project.

In [ ]:
original_project_labels = {label.name for label in project.pipeline.get_labels_per_task()[0]}
imported_project_labels = {label.name for label in imported_project.pipeline.get_labels_per_task()[0]}

if original_project_labels == imported_project_labels:
    print("Labels match between original and imported project.")
else:
    raise ValueError(
        f"Labels do not match between original and imported project."
        f"Original labels: {original_project_labels}, "
        f"Imported labels: {imported_project_labels}"
    )